In [1]:
!pip install tensorflow-recommenders
!pip install tensorflow-hub



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import csv
# Read the form responses CSV file and store the data into a list of tuples
data_list = []
with open('form_responses.csv', 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        key = (int(row[0]), row[1], row[2], tuple(row[3].split(";")), tuple(row[4].split(";")), row[5])
        value_list = [row[i:i+5] for i in range(6, len(row), 5)]
        data_list.append((key, value_list))

# Convert the list of tuples into a list of dictionaries
reconstructed_data = []
for i in range(0, len(data_list), 10):  # Assuming each dataset has 10 items
    dataset_dict = dict(data_list[i:i+10])
    reconstructed_data.append(dataset_dict)


In [3]:
from collections import defaultdict
import random

(updated_data, user_id_counter, user_id_mapping, stim_id_counter, stim_id_mapping) = (defaultdict(list), 1, {}, 1, {})

tlength = 0
for i in reconstructed_data:
    tlength += len(i)
user_ids = list(range(1, tlength + 1))
random.shuffle(user_ids)

def gendat(datadict):
    global updated_data, user_id_counter, user_id_mapping, stim_id_counter, stim_id_mapping
    for user, interactions in datadict.items():
        user = list(user)
        user[0] = int(user[0])
        user[5] = int(user[5])
        user = tuple(user)

        if user not in user_id_mapping:
            user_id_mapping[user] = user_id_counter
            user_id_counter += 1

        user_id = user_id_mapping[user]

        for interaction in interactions:
            stim = interaction[0]

            if stim not in stim_id_mapping:
                stim_id_mapping[stim] = stim_id_counter
                stim_id_counter += 1

            stim_id = stim_id_mapping[stim]

            updated_interaction = [stim_id] + [int(interaction[1])] + [int(interaction[2])] + [int(interaction[3])] + [int(interaction[4])]

            updated_data[user_id].append(updated_interaction)

for i in reconstructed_data:
    gendat(i)

In [4]:
import csv
stimdata=[]
new_stim_id_mapping={}
with open('Updated_StimData.csv','r',newline='') as f:
  reader = csv.reader(f, delimiter=',')
  for row in reader:
    stimdata.append(row)
for i in stimdata:
  new_stim_id_mapping[int(i[0])]= (i[1],i[2],i[3])
print(new_stim_id_mapping)

{1: ('ASMR', 'I experience ASMR as an intense tingling sensation primarily in my scalp and neck, triggered by specific sounds or visuals like whispers, tapping and scratching.', '1'), 2: ('Clay', 'I like to fidget with clay because of the tactile sensations it creates in my hands', '1'), 4: ('Interactive story-line games', 'Sometimes when I am really engaged and focused on playing an interactive story-line game, I tend to bite on my fingers or nails as a stim. ', '1'), 5: ('Imaginary air calligraphy', "Imaginary air calligraphy is a stim that involves moving one's finger in the air as if they are writing. It's like writing something in the invisible air.", '1'), 6: ('Vigorous leg shaking', 'When I am anxious or overwhelmed, I shake my legs vigorously to release the excess energy.', '1'), 7: ('Random vigorous unruly dancing', 'I perform random vigorous and unruly dancing, which mostly involves moving my limbs and hips quickly in different directions.', '1'), 8: ('Leg shaking', 'Leg shak

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import sklearn
from typing import Dict, Text
import os
import pprint
import tempfile

In [6]:
stims_df = pd.DataFrame(stim_id_mapping.items(), columns=['name', 'stim_id'])
users_df = pd.DataFrame(user_id_mapping.keys(), columns=['age', 'gender', 'sexuality', 'neurodivergent_conditions', 'hobbies', 'stimming_essentiality_score'])
users_df['user_id'] = user_id_mapping.values()
interaction_list = []
for user_id, interactions in updated_data.items():
    for interaction in interactions:
        interaction_list.append([user_id] + interaction)

interactions_df = pd.DataFrame(interaction_list, columns=['user_id', 'stim_id', 'usefulness_score', 'frequency_score', 'sensory_regulation_score', 'discomfort_score'])
print(len(users_df['user_id'].values))
users_df['neurodivergent_conditions'] = [np.array(x) if isinstance(x, (list, tuple)) else np.array([]) for x in users_df['neurodivergent_conditions']]
stims_df['description'] = stims_df['stim_id'].apply(lambda x: new_stim_id_mapping.get(x, " ")[1])
stims_df['harmfulness_score'] = stims_df['stim_id'].apply(lambda x: int(new_stim_id_mapping.get(x, 1)[2]))

182


In [7]:
users_df

,age,gender,sexuality,neurodivergent_conditions,hobbies,stimming_essentiality_score,user_id
0,20,Non-binary,Lesbian,"[Autism, PTSD]","(Sketching, Singing, Biking)",3,1
1,19,Female,Straight,"[ADHD, Generalized Anxiety Disorder, Panic Dis...","(Writing, Poetry, Painting, Birding, Designing)",3,2
2,19,Prefer not to say,Queer,"[ADHD, Generalized Anxiety Disorder, Social An...","(Reading, Painting, Video games)",4,3
3,20,Female,Bisexual,"[Social Anxiety Disorder, Depression, Epilepsy]","(Drawing, Music)",5,4
4,21,Prefer not to say,Asexual,"[Autism, ADHD, Depression, Borderline Personal...","(Reading, Sketching, Binge reading)",4,5
...,...,...,...,...,...,...,...
177,19,Prefer not to say,Pansexual,"[ADHD, Eating Disorder]","(Dancing, Baking, Vlogging)",2,178
178,23,Female,Straight,"[OCD, PTSD, Depression]","(Yoga, Meditation, Programming)",5,179
179,20,Non-binary,Demisexual,"[Generalized Anxiety Disorder, PTSD]","(Knitting, Watching movies, Video games)",3,180
180,22,Male,Bisexual,"[Autism, ADHD, Anxiety]","(Music, Running, Photography)",4,181


In [8]:
def score_calculator(interaction):
  harm_score=(stims_df[stims_df['stim_id']== interaction[0]])['harmfulness_score']
  return int(2 * interaction[1] + 2 * interaction[2] + 4 * interaction[3] - interaction[4] - 2.5 * (harm_score))

In [10]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [11]:
def generate_embeddings(column_data):
    return embed(tf.convert_to_tensor(column_data, dtype=tf.string)).numpy()

def tensorize(column_data):
    return tf.constant(column_data, dtype=tf.float32)

# Preprocess users_df
users_df['hobbies_embedding'] = users_df['hobbies'].apply(lambda x: generate_embeddings([", ".join(x)]))
users_df['conditions_embedding'] = users_df['neurodivergent_conditions'].apply(lambda x: generate_embeddings([" and ".join(x)]))

users_df['hobbies_embedding'] = users_df['hobbies_embedding'].apply(tensorize)
users_df['conditions_embedding'] = users_df['conditions_embedding'].apply(tensorize)

# Preprocess stims_df
stims_df['name_embedding'] = stims_df['name'].apply(lambda x: generate_embeddings([x]))
stims_df['description_embedding'] = stims_df['description'].apply(lambda x: generate_embeddings([x]))

stims_df['name_embedding'] = stims_df['name_embedding'].apply(tensorize)
stims_df['description_embedding'] = stims_df['description_embedding'].apply(tensorize)


In [12]:
# Merge the interaction and stim DataFrames to get the stim names and ratings
interactions_with_stims = pd.merge(interactions_df, stims_df, on='stim_id')
# Modify the user_stims DataFrame creation to include only description_embeddings of stims with scores greater than 20
user_stims = interactions_with_stims.groupby('user_id').apply(lambda x: x[['stim_id', 'usefulness_score', 'frequency_score', 'sensory_regulation_score', 'discomfort_score']].apply(lambda y: stims_df[stims_df['stim_id'] == y['stim_id']]['description_embedding'].values[0][0] if score_calculator([y['stim_id'], y['usefulness_score'], y['frequency_score'], y['sensory_regulation_score'], y['discomfort_score']]) > 20 else None, axis=1).dropna().tolist()).reset_index(name='current_stims')
# Merge the user_stims DataFrame with the users_df DataFrame to add the current_stims feature
users_df = pd.merge(users_df, user_stims, on='user_id', how='left')
# Convert the 'current_stims' column to a tensor with shape (N, 512)
users_df['current_stims'] = users_df['current_stims'].apply(lambda x: tf.stack(x) if len(x) > 0 else None)
users_df['current_stims'] = users_df['current_stims'].apply(lambda x: x[:4] if ((x is not None) and (len(x)>4) ) else x)


In [13]:
len(users_df['current_stims'][11])

2

In [14]:
# Create a dictionary to keep track of the associated conditions and hobbies for each stim
associated_conditions_hobbies = {stim_id: {'combined': []} for stim_id in stims_df['stim_id'].values}

# Iterate over each user's interactions
for user_id, interactions in updated_data.items():
    user_data = users_df.loc[users_df['user_id'] == user_id]
    conditions = ", ".join(user_data['neurodivergent_conditions'].values[0])
    hobbies = " and ".join(user_data['hobbies'].values[0])
    conditions_embedding = generate_embeddings([conditions])[0]
    hobbies_embedding = generate_embeddings([hobbies])[0]
    
    for stim_id, usefulness_score, frequency_score, sensory_regulation_score, discomfort_score in interactions:
        # Get the associated_conditions_hobbies dictionary for this stim
        stim_dict = associated_conditions_hobbies[stim_id]
        # Combine the conditions and hobbies for the current user and add them to the stim_dict
        stim_dict['combined'].append(tf.constant(conditions_embedding, dtype=tf.float32))
        stim_dict['combined'].append(tf.constant(hobbies_embedding, dtype=tf.float32))

# Now that we have the associated conditions and hobbies for each stim, add them to the stims_df
stims_df['associated_conditions_hobbies'] = stims_df['stim_id'].apply(lambda x: tf.stack(associated_conditions_hobbies[x]['combined']))


In [15]:
tf.keras.backend.set_floatx('float32')

In [16]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = "logs"
tensorboard_callback = TensorBoard(log_dir=log_dir, update_freq='batch')


In [17]:
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Flatten, Reshape, Concatenate,BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, Normalization, TextVectorization
tf.random.set_seed(42)
np.random.seed(42)
from tensorflow.keras.constraints import Constraint

class WeightClip(Constraint):
    def __init__(self, clip_value):
        self.clip_value = clip_value

    def __call__(self, w):
        return tf.clip_by_value(w, -self.clip_value, self.clip_value)

    def get_config(self):
        return {'clip_value': self.clip_value}

# Preprocessing layers
gender_lookup = StringLookup()
gender_lookup.adapt(users_df['gender'])

sexuality_lookup = StringLookup()
sexuality_lookup.adapt(users_df['sexuality'])
'''
nd_condition_lookup = StringLookup()
nd_condition_lookup.adapt(np.unique(np.concatenate(users_df['neurodivergent_conditions'].values)))'''

age_normalizer = Normalization()
age_normalizer.adapt(users_df['age'])

score_normalizer = Normalization()
score_normalizer.adapt(users_df['stimming_essentiality_score'])

# TextVectorization layer for stim names
max_tokens = 2000  # Maximum number of unique tokens, adjust this value according to your dataset
output_sequence_length = 16 # Adjust this value based on the expected number of tokens per stim_name
embedding_dim = 32  # Dimension of the dense vectors

# Create a Normalization layer for harmfulness_score
harmfulness_score_normalizer = Normalization()
harmfulness_score_normalizer.adapt(stims_df['harmfulness_score'])


regul=5e-5   #regularization
def create_user_input_layers():
    age_input = tf.keras.Input(shape=(1,), name="age", dtype=tf.int32)
    gender_input = tf.keras.Input(shape=(1,), name="gender", dtype=tf.string)
    sexuality_input = tf.keras.Input(shape=(1,), name="sexuality", dtype=tf.string)
    neurodivergent_conditions_input = tf.keras.Input(shape=(1,512) , name="nd_conditions_embedding", dtype=tf.float32)
    hobbies_embedding_input = tf.keras.Input(shape=(1,512) , name="hobbies_embedding", dtype=tf.float32)
    stimming_essentiality_score_input = tf.keras.Input(shape=(1,), name="stimming_essentiality_score", dtype=tf.int32)
    current_stims_input = tf.keras.Input(shape=(None, 512), name="current_stims", dtype=tf.float32)
    return {
        "age": age_input,
        "gender": gender_input,
        "sexuality": sexuality_input,
        "nd_conditions_embedding": neurodivergent_conditions_input,
        "hobbies_embedding": hobbies_embedding_input,
        "stimming_essentiality_score": stimming_essentiality_score_input,
        "current_stims": current_stims_input,
    }
def create_stim_input_layers():
    name_input = tf.keras.Input(shape=(1,512), name="name_embedding", dtype=tf.float32)
    description_input = tf.keras.Input(shape=(1,512), name="description_embedding", dtype=tf.float32)
    harmfulness_score_input = tf.keras.Input(shape=(1,), name="harmfulness_score", dtype=tf.int32)
    associated_conditions_hobbies_input= tf.keras.Input(shape=(2, 512), dtype=tf.float32, name="associated_conditions_hobbies")
    return {"name_embedding": name_input, "description_embedding": description_input, "harmfulness_score": harmfulness_score_input, 
            'associated_conditions_hobbies':associated_conditions_hobbies_input}

def create_user_model(user_input_layers):
    clip_value = 0.2
    clip_constraint = WeightClip(clip_value)
    # Preprocessing layers
    age_normalized = tf.expand_dims(age_normalizer(user_input_layers["age"]), -1)
    gender_embedded = gender_lookup(user_input_layers["gender"])
    sexuality_embedded = sexuality_lookup(user_input_layers["sexuality"])
    neurodivergent_conditions_embedded = user_input_layers["nd_conditions_embedding"]
    hobbies_embedded = user_input_layers["hobbies_embedding"]
    stimming_essentiality_score_normalized = tf.expand_dims(score_normalizer(user_input_layers["stimming_essentiality_score"]),-1)
    current_stims_embeddings = user_input_layers["current_stims"]

    # Embedding layers
    embedding_dim = 8
    gender_embedding = Embedding(input_dim=len(gender_lookup.get_vocabulary()), output_dim=embedding_dim)(gender_embedded)
    sexuality_embedding = Embedding(input_dim=len(sexuality_lookup.get_vocabulary()), output_dim=embedding_dim)(sexuality_embedded)
    #neurodivergent_conditions_dense = Embedding(input_dim=len(nd_condition_lookup.get_vocabulary()), output_dim=embedding_dim)(neurodivergent_conditions_embedded)
    
    
    # Dense layers
    age_dense = Dense(32, kernel_initializer='random_normal',activation='swish', kernel_constraint=clip_constraint,kernel_regularizer=tf.keras.regularizers.l1(regul))(tf.reshape(age_normalized, (-1, 1)))
    stimming_essentiality_dense = Dense(128, kernel_initializer='random_normal',kernel_regularizer=tf.keras.regularizers.l1(regul),activation='swish', kernel_constraint=clip_constraint)(tf.reshape(stimming_essentiality_score_normalized, (-1, 1)))

    gender_dense = Dense(32, kernel_initializer='random_normal',activation=tf.keras.activations.selu,kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(Flatten()(gender_embedding))
    gender_dense = tf.debugging.check_numerics(gender_dense, message="NaN or Inf found in gender_dense")
    
    sexuality_dense = Dense(128, kernel_initializer='random_normal',activation='swish',kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(Flatten()(sexuality_embedded))
    sexuality_dense = tf.debugging.check_numerics(sexuality_dense, message="NaN or Inf found in sexuality_dense")
    
    #neurodivergent_conditions_pooled = GlobalAveragePooling1D()(neurodivergent_conditions_dense)
    nd_conditions_normalizer=BatchNormalization()
    nd_condition_dense = nd_conditions_normalizer(Dense(2048, kernel_initializer='random_normal',kernel_regularizer=tf.keras.regularizers.l1(regul),activation='swish', kernel_constraint=clip_constraint)(Flatten()(neurodivergent_conditions_embedded)))
    nd_condition_dense = tf.debugging.check_numerics(nd_condition_dense, message="NaN or Inf found in nd_condition_dense")
    
    hobbies_normalizer=BatchNormalization()
    hobbies_dense = hobbies_normalizer(Dense(2048, kernel_initializer='random_normal',kernel_regularizer=tf.keras.regularizers.l1(regul),activation='swish', kernel_constraint=clip_constraint)(Flatten()(hobbies_embedded)))
    hobbies_dense = tf.debugging.check_numerics(hobbies_dense, message="NaN or Inf found in hobbies_dense")
    
    avg_pooled_stims = tf.reduce_mean(current_stims_embeddings, axis=1)
    stim_normalizer=BatchNormalization()
    current_stims_embeddings_dense = stim_normalizer(Dense(1024, kernel_initializer='random_normal',kernel_regularizer=tf.keras.regularizers.l1(regul),activation='swish', kernel_constraint=clip_constraint)(avg_pooled_stims))
    current_stims_embeddings_dense = tf.debugging.check_numerics(current_stims_embeddings_dense, message="NaN or Inf found in current_stims_embeddings_dense")

    flattened_stims_dense=Flatten()(current_stims_embeddings_dense)

    # Concatenate dense layers
    concatenated = Concatenate(axis=-1)([
        age_dense,
        gender_dense,
        sexuality_dense,
        nd_condition_dense,
        hobbies_dense,
        stimming_essentiality_dense,
        current_stims_embeddings_dense
    ])

    flat_embeddings = Flatten()(concatenated)
    # User Dense layers
    layer1_norm=BatchNormalization()
    dense_1 = layer1_norm(Dense(8192, kernel_initializer='random_normal',activation='swish', kernel_constraint=clip_constraint,kernel_regularizer=tf.keras.regularizers.l1(regul))(flat_embeddings))
    layer2_norm=BatchNormalization()
    dense_2 = layer2_norm(Dense(4096, kernel_initializer='random_normal',activation='swish', kernel_constraint=clip_constraint,kernel_regularizer=tf.keras.regularizers.l1(regul))(dense_1))
    layer3_norm=BatchNormalization()
    dense_3 = layer3_norm(Dense(2048, kernel_initializer='random_normal',activation='swish',kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(dense_2))
    layer4_norm=BatchNormalization()
    dense_4 = layer4_norm(Dense(1024, kernel_initializer='random_normal',activation='swish',kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(dense_3))
    model = tf.keras.Model(inputs=user_input_layers, outputs=dense_4, name="user_model")
    return model

def create_stim_model(stim_input_layers):
    clip_value = 0.2
    clip_constraint = WeightClip(clip_value)
    embedded_name = stim_input_layers["name_embedding"]
    description_embedded = stim_input_layers["description_embedding"]

    harmfulness_score_normalized = tf.expand_dims(harmfulness_score_normalizer(stim_input_layers["harmfulness_score"]), -1)
    associated_conditions_hobbies_embedded= stim_input_layers["associated_conditions_hobbies"]

    name_dense=Dense(1024, kernel_initializer='random_normal',activation='swish', kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(Flatten()(embedded_name))
    description_dense=Dense(1024, kernel_initializer='random_normal',activation='swish', kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(Flatten()(description_embedded))
    harmfulness_score_dense=Dense(16, kernel_initializer='random_normal',activation=tf.keras.activations.selu, kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(tf.reshape(harmfulness_score_normalized,(-1,1)))
    avg_pooled_hobbies = tf.reduce_mean(associated_conditions_hobbies_embedded, axis=1)
    hobby_norm=BatchNormalization()
    associated_conditions_hobbies_embedded_dense = hobby_norm(Dense(2048, kernel_initializer='random_normal', kernel_regularizer=tf.keras.regularizers.l1(regul),activation='swish', kernel_constraint=clip_constraint)(avg_pooled_hobbies))
    flattened_hobbies_dense = Flatten()(associated_conditions_hobbies_embedded_dense)

    concatenated = Concatenate(axis=-1)([
        name_dense,
        description_dense,
        harmfulness_score_dense,
        flattened_hobbies_dense
    ])
    flat_embeddings = Flatten()(concatenated)
    # Flatten and Dense layers
    layer1_norm=BatchNormalization()
    dense_1 = layer1_norm(Dense(8192, kernel_initializer='random_normal',activation='swish',kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(flat_embeddings))
    layer2_norm=BatchNormalization()
    dense_2 = layer2_norm(Dense(4096, kernel_initializer='random_normal', activation='selu',kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(dense_1))
    layer3_norm=BatchNormalization()
    dense_3 = layer3_norm(Dense(2048, kernel_initializer='random_normal',activation='swish', kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)(dense_2))
    layer4_norm=BatchNormalization()
    dense_4 = layer4_norm(Dense(1024, kernel_initializer='random_normal', kernel_regularizer=tf.keras.regularizers.l1(regul),activation='selu', kernel_constraint=clip_constraint)(dense_3))
    return tf.keras.Model(inputs=stim_input_layers, outputs=dense_4, name="stim_model")

user_input_layers = create_user_input_layers()
stim_input_layers = create_stim_input_layers()
user_model = create_user_model( user_input_layers)
stim_model = create_stim_model( stim_input_layers)

In [18]:
relug=5e-5
class ScoreEstimator(tf.keras.Model):
    def __init__(self, user_model, stim_model):
        super().__init__()
        self.user_model = user_model
        self.stim_model = stim_model
        self.concat_layer = Concatenate(axis=-1)
        clip_value = 0.5
        clip_constraint = WeightClip(clip_value)

        self.dense_1a = Dense(4096, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_1b = Dense(2048, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_2a = Dense(512, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_2b = Dense(256, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_3a = Dense(128, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_3b = Dense(64, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_4a = Dense(32, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_4b = Dense(16, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_5a = Dense(8, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.dense_5b = Dense(4, activation='swish', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1(relug), kernel_constraint=clip_constraint)
        self.output_layer = Dense(1, activation='swish', kernel_regularizer=tf.keras.regularizers.l1(regul), kernel_constraint=clip_constraint)

        self.batch_norm1 = BatchNormalization()
        self.batch_norm2 = BatchNormalization()
        self.batch_norm3 = BatchNormalization()
        self.batch_norm4 = BatchNormalization()
        self.batch_norm5 = BatchNormalization()
        
        self.loss_fn = tf.keras.losses.MeanSquaredError()

    def call(self, inputs, training=False):
        user_inputs, stim_inputs = inputs
        user_output = self.user_model(user_inputs)
        stim_output = self.stim_model(stim_inputs)
        concatenated = self.concat_layer([user_output, stim_output])
        dense_1a_output = self.batch_norm1(self.dense_1a(concatenated))
        dense_1b_output = self.dense_1b(dense_1a_output)
        dense_2a_output = self.batch_norm2(self.dense_2a(dense_1b_output))
        dense_2b_output = self.dense_2b(dense_2a_output)
        dense_3a_output = self.batch_norm3(self.dense_3a(dense_2b_output))
        dense_3b_output = self.dense_3b(dense_3a_output)
        dense_4a_output = self.batch_norm4(self.dense_4a(dense_3b_output))
        dense_4b_output = self.dense_4b(dense_4a_output)
        dense_5a_output = self.batch_norm5(self.dense_5a(dense_4b_output))
        dense_5b_output = self.dense_5b(dense_5a_output)
        output = self.output_layer(dense_5b_output)
        return output
    '''
    def compute_loss(self, inputs, scores, training=False):
        user_inputs, stim_inputs = inputs
        predictions = self.call((user_inputs, stim_inputs), training=training)
        loss = self.loss_fn(scores, predictions)
        return loss

    def train_step(self, data):
        inputs, scores = data
        with tf.GradientTape() as tape:
            loss = self.compute_loss(inputs, scores, training=True)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

    def test_step(self, data):
        inputs, scores = data
        loss = self.compute_loss(inputs, scores, training=False)
        return {"loss": loss}
    def fit(self, inputs, scores, epochs=1, batch_size=256, validation_split=0.0):
        dataset = tf.data.Dataset.from_tensor_slices((inputs, scores)).batch(batch_size).shuffle(buffer_size=1000)
        if validation_split > 0.0:
            num_val_samples = int(validation_split * len(scores))
            num_train_samples = len(scores) - num_val_samples
            train_dataset = dataset.take(num_train_samples)
            val_dataset = dataset.skip(num_train_samples)
        else:
            train_dataset = dataset
            val_dataset = None

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            step = 0
            for step_inputs, step_scores in train_dataset:
                step_loss = self.train_step((step_inputs, step_scores))["loss"]
                step += 1
                print(f"Step {step}, loss: {step_loss}")

            if val_dataset is not None:
                val_losses = []
                for val_inputs, val_scores in val_dataset:
                    val_loss = self.test_step((val_inputs, val_scores))["loss"]
                    val_losses.append(val_loss)
                print(f"Validation loss: {np.mean(val_losses)}")
      '''


In [19]:
class RankingModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()

    self.ranking_model: tf.keras.Model = ScoreEstimator(user_model,stim_model)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, inputs):
      #tensor_user_inputs, tensor_stim_inputs, scores = inputs[0]
      return self.ranking_model((inputs['user_inputs'], inputs['stim_inputs']))

  def compute_loss(self, inputs, training=False) -> tf.Tensor:
      rating_predictions = self(inputs)
      # The task computes the loss and the metrics.
      return self.task(labels=inputs['labels'], predictions=rating_predictions)

In [20]:
model = RankingModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=3e-4,clipvalue=0.5))


In [21]:
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.model_selection import train_test_split

np_config.enable_numpy_behavior()


# Split interactions_df into train and test sets
train_interactions_df, test_interactions_df = train_test_split(interactions_df, test_size=0.2, random_state=42)

def pad_or_truncate(arr, target_shape):
    if arr.shape[0] < target_shape[0]:
        padding = np.zeros((target_shape[0] - arr.shape[0], target_shape[1]))
        arr = np.vstack((arr, padding))
    elif arr.shape[0] > target_shape[0]:
        arr = arr[:target_shape[0], :]
    return arr
def score_calculate(interaction):
  harm_score=(stims_df[stims_df['stim_id']== interaction['stim_id']])['harmfulness_score']
  return int(2 * interaction['usefulness_score'] + 2 * interaction['frequency_score'] + 4 * interaction['sensory_regulation_score'] - interaction['discomfort_score'] - 2.5 * (harm_score))

def preprocess_data(interactions_df, users_df, stims_df, score_calculator):
    user_inputs_list = []
    stim_inputs_list = []
    scores_list = []

    for index, interaction in interactions_df.iterrows():
        user_id = interaction['user_id']
        stim_id = interaction['stim_id']
        user = users_df[users_df['user_id'] == user_id].iloc[0]
        stim = stims_df[stims_df['stim_id'] == stim_id].iloc[0]

        # Extract user_inputs
        user_inputs = {
            'age': user['age'],
            'gender': user['gender'],
            'sexuality': user['sexuality'],
            'nd_conditions_embedding': user['conditions_embedding'],
            'hobbies_embedding': user['hobbies_embedding'],
            'stimming_essentiality_score': user['stimming_essentiality_score'],
            'current_stims': user['current_stims']
        }

        # Extract stim_inputs
        stim_inputs = {
            'name_embedding': stim['name_embedding'],
            'description_embedding': stim['description_embedding'],
            'harmfulness_score': stim['harmfulness_score'],
            'associated_conditions_hobbies': stim['associated_conditions_hobbies']
        }

        # Calculate score
        score = score_calculate(interaction)
        scores_list.append(score)

        user_inputs_list.append(user_inputs)
        stim_inputs_list.append(stim_inputs)
        
    # Convert data to tensors (except for RaggedTensors and 'associated_conditions_hobbies')
    tensor_user_inputs = {key: tf.convert_to_tensor(np.array([user_input[key] for user_input in user_inputs_list])) for key in user_inputs_list[0] if key not in ['current_stims']}
    tensor_stim_inputs = {key: tf.convert_to_tensor(np.array([stim_input[key] for stim_input in stim_inputs_list])) for key in stim_inputs_list[0] if key != 'associated_conditions_hobbies'}

    # Create a list of lists for 'current_stims'
    #max_current_stims_length = max([len(user_input['current_stims']) if user_input['current_stims'] is not None else 0 for user_input in user_inputs_list])
    max_current_stims_length = 3
    current_stims_list = []
    for user_input in user_inputs_list:
        if user_input['current_stims'] is not None:
            current_stims_list.append(user_input['current_stims'].numpy().tolist())
        else:
            current_stims_list.append([])
    tensor_user_inputs['current_stims'] = tf.ragged.constant(current_stims_list, dtype=tf.float32).to_tensor(default_value=0.0)

    # Handle 'associated_conditions_hobbies' separately
    tensor_stim_inputs['associated_conditions_hobbies'] = tf.stack([tf.convert_to_tensor(pad_or_truncate(stim_input['associated_conditions_hobbies'], (2, 512)), dtype=tf.float32) for stim_input in stim_inputs_list], axis=0)

    tensor_scores = tf.convert_to_tensor(np.array(scores_list), dtype=tf.float32)

    return (tensor_user_inputs, tensor_stim_inputs, tensor_scores)

(tensor_user_inputs_train, tensor_stim_inputs_train, tensor_scores_train) = preprocess_data(train_interactions_df, users_df, stims_df, score_calculator)
(tensor_user_inputs_test, tensor_stim_inputs_test, tensor_scores_test) = preprocess_data(test_interactions_df, users_df, stims_df, score_calculator)

# Get a sample batch of data
sample_user_inputs = {key: tensor[:1] for key, tensor in tensor_user_inputs_train.items()}
sample_stim_inputs = {key: tensor[:1] for key, tensor in tensor_stim_inputs_train.items()}

# Add the missing 'current_stims' input
sample_user_inputs['current_stims'] = tensor_user_inputs_train['current_stims'][:1]

# Call the model with the sample batch to build the graph
_ = model({'user_inputs' : sample_user_inputs, 'stim_inputs': sample_stim_inputs, 'labels': 0})


# Train the model
epochs = 25
model.fit({ 'user_inputs' : tensor_user_inputs_train, 'stim_inputs': tensor_stim_inputs_train, 'labels': tensor_scores_train}, epochs=epochs,steps_per_epoch=331, callbacks=[tensorboard_callback])

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
model.summary()

In [39]:
# Evaluate the model on the test set
model.task.metrics.append(tf.keras.metrics.MeanAbsoluteError())
model.evaluate({'user_inputs' : tensor_user_inputs_test, 'stim_inputs': tensor_stim_inputs_test, 'labels': tensor_scores_test})

5/5 [==============================] - 2s 219ms/step - root_mean_squared_error: 6.1068 - loss: 36.8775 - regularization_loss: 361.0368 - total_loss: 397.9143


[6.106773853302002, 35.941497802734375, 361.03680419921875, 396.9783020019531]

In [40]:
# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def text_embedder(text):
    return embed(text)

def create_prediction_input(user_data, stim_data):
    user_inputs = {
        "age": tf.constant([user_data["age"]], dtype=tf.int32),
        "gender": tf.constant([user_data["gender"]], dtype=tf.string),
        "sexuality": tf.constant([user_data["sexuality"]], dtype=tf.string),
        "nd_conditions_embedding": tf.expand_dims(tf.expand_dims(text_embedder([" and ".join(user_data["neurodivergent_conditions"])])[0], axis=0), axis=1),
        "hobbies_embedding": tf.expand_dims(tf.expand_dims(text_embedder([user_data["hobbies"]])[0], axis=0), axis=1),
        "stimming_essentiality_score": tf.constant([user_data["stimming_essentiality_score"]], dtype=tf.int32),
        "current_stims": tf.expand_dims(tf.expand_dims(text_embedder([user_data["current_stims"]])[0], axis=0), axis=1),
    }

    stim_inputs = {
        "name_embedding": tf.expand_dims(tf.expand_dims(text_embedder([stim_data["name"]])[0], axis=0), axis=1),
        "description_embedding": tf.expand_dims(tf.expand_dims(text_embedder([stim_data["description"]])[0], axis=0), axis=1),
        "harmfulness_score": tf.constant([stim_data["harmfulness_score"]], dtype=tf.int32),
        "associated_conditions_hobbies": tf.expand_dims(pad_or_truncate(text_embedder(stim_data["associated_conditions_hobbies"]), (2, 512)), axis=0),
    }


    return user_inputs, stim_inputs

user_data = {
    "age": 20,
    "gender": "Female",
    "sexuality": "Bisexual",
    "neurodivergent_conditions": ['Social Anxiety Disorder', 'Depression','Epilepsy'],
    "hobbies": "Drawing, Music",
    "stimming_essentiality_score": 5,
    "current_stims": "Tapping Feet",
}

stim_data = {
    "name": "Fidgeting hands",
    "description": "I start tearing random bits of paper whenever I'm mindless",
    "harmfulness_score": 1,
    "associated_conditions_hobbies": ["Dyslexia", "Masturbation"],
}


user_inputs, stim_inputs = create_prediction_input(user_data, stim_data)

# Make a prediction
prediction = model({ 'user_inputs' : user_inputs, 'stim_inputs': stim_inputs, 'labels': 10})
print(prediction)

tf.Tensor([[25.322544]], shape=(1, 1), dtype=float32)


In [ ]:
%load_ext tensorboard
log_folder = 'logs'
%reload_ext tensorboard

In [ ]:
tensorboard --logdir logs

In [23]:
# Save the prediction model
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.save("stimming_recommender")
# Save the model's weights
model.save_weights("stimming_recommender_weights/weights.ckpt")
user_model.save_weights("user_model_weights/weights.ckpt")
stim_model.save_weights("stim_model_weights/weights.ckpt")

In [42]:
#Restore to use model
def create_recommender_model(user_input_layers, stim_input_layers):
    user_model = create_user_model(user_input_layers)
    stim_model = create_stim_model(stim_input_layers)
    recommender_model = RankingModel()
    recommender_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4))
    return recommender_model
user_input_layers = create_user_input_layers()
stim_input_layers = create_stim_input_layers()

# Create the model architecture
loaded_model = create_recommender_model(user_input_layers, stim_input_layers)

# Load the saved weights
checkpoint = tf.train.Checkpoint(model=loaded_model)
checkpoint.restore("stimming_recommender/weights.ckpt-1").expect_partial()


In [44]:
# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def text_embedder(text):
    return embed(text)

def create_prediction_input(user_data, stim_data):
    user_inputs = {
        "age": tf.constant([user_data["age"]], dtype=tf.int32),
        "gender": tf.constant([user_data["gender"]], dtype=tf.string),
        "sexuality": tf.constant([user_data["sexuality"]], dtype=tf.string),
        "nd_conditions_embedding": tf.expand_dims(tf.expand_dims(text_embedder([" and ".join(user_data["neurodivergent_conditions"])])[0], axis=0), axis=1),
        "hobbies_embedding": tf.expand_dims(tf.expand_dims(text_embedder([user_data["hobbies"]])[0], axis=0), axis=1),
        "stimming_essentiality_score": tf.constant([user_data["stimming_essentiality_score"]], dtype=tf.int32),
        "current_stims": tf.expand_dims(tf.expand_dims(text_embedder([user_data["current_stims"]])[0], axis=0), axis=1),
    }

    stim_inputs = {
        "name_embedding": tf.expand_dims(tf.expand_dims(text_embedder([stim_data["name"]])[0], axis=0), axis=1),
        "description_embedding": tf.expand_dims(tf.expand_dims(text_embedder([stim_data["description"]])[0], axis=0), axis=1),
        "harmfulness_score": tf.constant([stim_data["harmfulness_score"]], dtype=tf.int32),
        "associated_conditions_hobbies": tf.expand_dims(pad_or_truncate(text_embedder(stim_data["associated_conditions_hobbies"]), (2, 512)), axis=0),
    }


    return user_inputs, stim_inputs

user_data = {
    "age": 20,
    "gender": "Female",
    "sexuality": "Bisexual",
    "neurodivergent_conditions": ['Social Anxiety Disorder', 'Depression','Epilepsy'],
    "hobbies": "Drawing, Music",
    "stimming_essentiality_score": 5,
    "current_stims": "Tapping Feet",
}

stim_data = {
    "name": "Fidgeting hands",
    "description": "I start tearing random bits of paper whenever I'm mindless",
    "harmfulness_score": 1,
    "associated_conditions_hobbies": ["Dyslexia", "Masturbation"],
}


user_inputs, stim_inputs = create_prediction_input(user_data, stim_data)

# Make a prediction
prediction = loaded_model({ 'user_inputs' : user_inputs, 'stim_inputs': stim_inputs, 'labels': 10})
print(prediction)

tf.Tensor([[25.322544]], shape=(1, 1), dtype=float32)
